In [ ]:
using JuMP
using Ipopt
using Random
using LinearAlgebra
using ForwardDiff

∇(f, x) = ForwardDiff.gradient(f, x)
D(θ, λ) = ForwardDiff.derivative(θ, λ)

In [ ]:
function Armijo_ls(θ, λ, α, β, λbar) 
    θ₀  = θ(0)                                 # Function value at zero
    Dθ₀ = D(θ, 0)                              # Derivative (slope) at zero   
    while (θ(λ) > θ₀ + α*λ*Dθ₀) && (λ <= λbar) # Check termination condition
        λ = β*λ                                # Reduce λ until condition is satisfied
    end
    return λ
end

In [ ]:
function FW()
    
    Random.seed!(1)
    
    ## Generate random data
    M   = 1000
    N   = 100
    A   = rand(M,N)
    b   = rand(M)
    
    ## Constants
    c    = 1.0            # RHS of the constraint ||x||² ≤ c
    λbar = 1.0            # Upper bound of λ in FW method
    λ    = λbar           # Initial step size λ
    eps  = 1e-10          # Convergence tolerance
    
    ## Armijo parameters
    α   = 0.01
    β   = 0.70
    
    ## Objective function to be minimized
    f(x) = 0.5*dot(A*x - b, A*x - b)
    
    ## Initial values
    xᵏ  = 0.1*ones(N)
    
    ## Compute x̄ᵏ from x̄ᵏ = argmin{x ∈ ℜⁿ : ∇f(xᵏ)ᵀx, x ∈ S}    
    model = Model(with_optimizer(Ipopt.Optimizer, print_level = 1)) 
    @variable(model, x[1:N])
    
    #### TODO: Write the objective and constraints here
    @objective(model, Min, dot(∇(f, xᵏ), x))
    @constraint(model, dot(x, x,)≤c)

    optimize!(model)
    x̄ᵏ = value.(x)
    dᵏ = x̄ᵏ - xᵏ 
    
    ## Iteration counter + 1st solution
    k   = 1
    xᵏ  = xᵏ + λ*dᵏ  
    
    while dot(∇(f, xᵏ), dᵏ) > eps
        
        ## Compute x̄ᵏ from x̄ᵏ = argmin{x ∈ ℜⁿ : ∇f(xᵏ)ᵀx, x ∈ S}
        model = Model(with_optimizer(Ipopt.Optimizer, print_level = 0)) 
        @variable(model, x[1:N])
        
        #### TODO: Write the objective and constraints here
        @objective(model, Min, dot(∇(f, xᵏ), x))
        @constraint(model, dot(x, x,)≤c)
                
        optimize!(model)
        x̄ᵏ = value.(x)
        dᵏ = x̄ᵏ - xᵏ

        #### Line search
        θ(λ) = f(xᵏ + λ*dᵏ)
        λ    = Armijo_ls(θ, λ, α, β, λbar)
        
        #### Update solution
        k  = k + 1
        xᵏ = xᵏ + λ*dᵏ      
    end
    
    ## Get optimal cost
    obj = f(xᵏ)
    return (k, xᵏ, obj)
end

In [ ]:
## Solve model
(k, xsol, obj) = FW()

## Print solution
println("  Iterations: ", k)
println("Optimal cost: ", round(obj, digits = 2))

#### OPTIMAL COST SHOULD BE $\approx 36.71$ 
#### WITH $k = 280$ ITERATIONS